In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import heapq
import cv2
import pandas as pd
from tensorflow import keras
import numpy as np
import os
import json
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

# 0. Data Preprocessing

In [2]:
#Feature Normalization:
scaler = StandardScaler()

# 1. One-Feature-Model:   

Regard application swichting as the only feature, 
e.g. instagram --> whatsapp 
instagram as X_data
whatsapp as y_label


In [25]:
account_path = r"D:\Dokuments\Project_of_HCI\emva-dataset\EMVA_from_gDrive\f5d25f46-5445-4cb5-be13-4d3a3801c0fe"
input_raw = []
label_raw = []
temp_switchingTime = []
temp_switchingTimeDate = []
duration = []


for recording in os.listdir(account_path):
    recording_path = os.path.join (account_path, str(recording))
    
    assert(os.path.exists(recording_path))

    list_session = []
    for file in os.listdir(recording_path):
        if str(file)[:7] == "Session":
            list_session.append(str(file))

    #if list_session==[]:
    #    continue    
    
    with open (os.path.join(recording_path, "application_data.txt")) as application_file:
        application_data = json.load(application_file)["application_data"]
        application_index = 0
        while application_index < len(application_data):
            if (application_data[application_index]["name"] != "com.miui.home" 
            and application_data[application_index]["name"] != "ch.ethz.inf.vs.staals.visualattentionrecorder"):
                num = 1
                while application_index+num < len(application_data):
                    if (application_data[application_index+num]["name"] != "com.miui.home" 
                    and application_data[application_index+num]["name"] != "ch.ethz.inf.vs.staals.visualattentionrecorder"
                    and application_data[application_index+num]["name"] != application_data[application_index]["name"]):
                        input_raw.append (application_data[application_index]["name"])
                        label_raw.append (application_data[application_index+num]["name"])
                        temp_switchingTime.append (application_data[application_index+1]["time"])
                        temp_switchingTimeDate.append (application_data[application_index+1]["timeDate"])
                        duration.append (application_data[application_index+num]["time"] - application_data[application_index]["time"])
                        break
                    else:
                        num += 1
            
            application_index += 1

print("INPUT DATA:")
print(len(input_raw))
print(input_raw)
print("----------------------------------------------------------------")
print("LABEL DATA:")
print(len(label_raw))
print(label_raw)
print(temp_switchingTime)
feature_switchingTime = temp_switchingTime

INPUT DATA:
162
['com.android.settings', 'com.android.settings', 'com.instagram.android', 'com.miui.securitycenter', 'com.miui.cleanmaster', 'com.instagram.android', 'com.google.android.googlequicksearchbox', 'com.miui.securitycenter', 'com.whatsapp', 'com.google.android.youtube', 'com.instagram.android', 'com.instagram.android', 'com.spotify.music', 'com.whatsapp', 'com.google.android.gm', 'com.android.browser', 'com.whatsapp', 'com.google.android.youtube', 'com.android.settings', 'com.miui.securitycenter', 'com.google.android.youtube', 'com.spotify.music', 'com.Slack', 'com.android.settings', 'com.miui.securitycenter', 'com.instagram.android', 'com.miui.securitycenter', 'com.spotify.music', 'com.instagram.android', 'com.spotify.music', 'com.facebook.orca', 'com.spotify.music', 'com.miui.securitycenter', 'com.spotify.music', 'com.miui.securitycenter', 'com.android.providers.downloads.ui', 'ro.bcr.georgego', 'com.Slack', 'com.instagram.android', 'com.android.camera', 'com.eTokenBCR', '

In [4]:
app = []
for i in range(len(input_raw)):
    signal = 1
    for j in range(len(app)):
        if app[j]==input_raw[i]:
            signal = 0
            break 
    if signal!=0:
        app.append(input_raw[i])
for i in range(len(label_raw)):
    signal = 1
    for j in range(len(app)):
        if app[j]==label_raw[i]:
            signal = 0
            break 
    if signal!=0:
        app.append(label_raw[i])
input = []
for element in input_raw:
    input.append(app.index(element))
print(input)
label = []
for element in label_raw:
    label.append(app.index(element))
print(label)

[0, 0, 1, 2, 3, 1, 4, 2, 5, 6, 1, 1, 7, 5, 8, 9, 5, 6, 0, 2, 6, 7, 10, 0, 2, 1, 2, 7, 1, 7, 11, 7, 2, 7, 2, 12, 13, 10, 1, 14, 15, 1, 15, 7, 11, 7, 7, 1, 16, 7, 7, 1, 7, 1, 11, 5, 0, 1, 17, 7, 7, 11, 5, 5, 18, 18, 18, 11, 19, 20, 19, 19, 19, 19, 19, 0, 11, 1, 1, 1, 7, 0, 7, 14, 7, 14, 7, 7, 15, 7, 7, 7, 11, 7, 11, 7, 7, 11, 11, 11, 11, 11, 14, 21, 21, 21, 21, 14, 14, 21, 21, 14, 14, 14, 21, 14, 21, 14, 14, 21, 14, 21, 21, 14, 21, 14, 21, 14, 21, 0, 0, 22, 22, 22, 23, 5, 11, 0, 11, 5, 5, 7, 10, 7, 8, 24, 7, 7, 7, 7, 5, 1, 17, 17, 11, 7, 11, 7, 7, 11, 7, 7]
[1, 1, 2, 3, 1, 4, 2, 3, 6, 1, 7, 7, 5, 8, 9, 7, 6, 0, 2, 6, 7, 11, 0, 2, 1, 2, 7, 1, 8, 11, 7, 2, 7, 2, 12, 10, 10, 13, 11, 15, 7, 15, 7, 11, 7, 11, 0, 16, 5, 2, 1, 8, 1, 11, 1, 0, 1, 25, 11, 11, 11, 5, 26, 26, 11, 11, 11, 19, 20, 19, 0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0, 7, 14, 7, 14, 7, 0, 0, 21, 11, 11, 11, 7, 11, 7, 11, 11, 5, 14, 14, 14, 14, 21, 14, 14, 14, 14, 21, 21, 14, 14, 21, 21, 21, 14, 21, 14, 21, 21, 14, 21, 14, 14, 21, 14, 

In [5]:
feature_app = np.array(input)
np_label = np.array(label)
feature_app = feature_app.reshape(len(input_raw),1)
len(app)

27

In [6]:
feature_app.shape

(162, 1)

# from app(1 feature) to app

In [7]:
X = scaler.fit_transform(feature_app)
y = np_label
y.shape

(162,)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

## kNN Model:

In [9]:
kNN = KNeighborsClassifier(n_neighbors=7, weights='distance').fit(X_train,y_train) #probabilistic 
model_1_kNN = kNN.score(X_test, y_test)
print(model_1_kNN)

0.4634146341463415


# Random Forest Model:

In [10]:
rf = RandomForestClassifier(n_estimators=30)
rf.fit(X_train,y_train)
model_1_rf = rf.score(X_test, y_test)
print(model_1_rf)

0.43902439024390244


# 2. Model plus video features:

Video features are extracted from .csv file
Still with 3 different time windows

In [11]:
def extractFeatures(df_temp, switchingTime, sessionStartTime, timeWindow):
    df = df_temp.iloc[:,2:]
    index_timestamp_list=[]
    for index,timestamp in df["timestamp"].items():
        if timestamp >= (switchingTime-sessionStartTime)/1_000_000_000 - timeWindow and timestamp <= (switchingTime-sessionStartTime)/1_000_000_000:
            index_timestamp_list.append(index)

    df_timestamp_window = pd.DataFrame(columns=df.columns.values.tolist())
    for i in index_timestamp_list:
        df_timestamp_window=df_timestamp_window.append(df.iloc[i],ignore_index=True)

    index_confidence_list=[]
    for index,confidence in df_timestamp_window["confidence"].items():#use confidence or we could use "sucess==1" to check the validity
        if confidence > 0.8:
            index_confidence_list.append(index)

    df_window = pd.DataFrame(columns=df.columns.values.tolist())#contains the valid and high confidence features for all frames during the time window
    for i in index_confidence_list:
        df_window=df_window.append(df_timestamp_window.iloc[i],ignore_index=True)

    #df_window.mean()series form of all averaged value for the features during the time window
    video_mean_features = df_window.mean().values[3:11]#final features for one switch (291 video features totally)
    video_var_features = df_window.var().values[3:11]
    video_std_features = df_window.std().values[3:11]
    return video_mean_features, video_var_features, video_std_features

In [12]:
# video features in the past timeWindow=3/5/10 seconds
timeWindow = 3
list_mean_features_video = []
list_var_features_video = []
list_std_features_video = []
valid_index=[]

for recording in os.listdir(account_path):
    recording_path = os.path.join (account_path, str(recording))
    
    assert(os.path.exists(recording_path))

    print(recording)
    
    # open each of the recording file
    
    with open (os.path.join(recording_path, "video_data.txt")) as video_file:
        video_data = json.load(video_file)["video_data"]
    
        #num_session = len(video_data)
        #????????????????????????????????????????????? something wrong with the video_data for recording 001&002
        
        # count the number of sessions
        list_session = []
        for file in os.listdir(recording_path):
            if str(file)[:7] == "Session":
                list_session.append(str(file))
        
        if list_session==[]:
            continue
        
        
        # open each of the session file
        for session in list_session:
            session_path = os.path.join(recording_path, session)
            
            assert(os.path.exists(session_path))
            
            #get the timestamp which the video started recording
            sessionStartTime = video_data[int(session[-1])]["startTime"]
            sessionStopTime = video_data[int(session[-1])]["stopTime"]
            
            
            if os.path.exists((os.path.join(session_path, "r_video.csv")))==False:
                continue
            df_temp = pd.read_csv(os.path.join(session_path, "r_video.csv"))

            print(session)
  
            for switchingTime in feature_switchingTime:
                if switchingTime < sessionStopTime and switchingTime > sessionStartTime:
                    mean_video_features_switching, var_video_features_switching, std_video_features_switching = extractFeatures(df_temp, switchingTime, sessionStartTime, timeWindow)
                    if mean_video_features_switching.dtype=='float64' and var_video_features_switching.dtype=='float64' and std_video_features_switching.dtype=='float64':
                        valid_index.append(feature_switchingTime.index(switchingTime))
                        list_mean_features_video.append(mean_video_features_switching.tolist())
                        list_var_features_video.append(var_video_features_switching.tolist())
                        list_std_features_video.append(std_video_features_switching.tolist())
features_mean_video = np.array(list_mean_features_video)
features_var_video = np.array(list_var_features_video)
features_std_video = np.array(list_std_features_video)
print(valid_index)

Recording 000
Session 0000
Recording 001
Session 0000
Session 0001
Session 0002
Session 0003
Session 0004
Session 0005
Recording 002
Session 0000
Session 0001
Session 0002
Session 0003
Session 0004
Session 0005
Recording 003
Session 0000
Session 0001
Session 0002
Recording 004
Session 0000
Recording 005
Session 0000
Recording 006
Session 0000
Recording 007
Session 0000
Recording 008
Session 0000
Recording 009
Session 0000
Session 0001
Recording 010
Session 0000
Recording 011
Session 0000
Recording 012
Session 0000
Session 0001
Recording 013
Session 0000
Recording 014
Session 0000
Recording 015
Session 0000
Recording 016
Session 0000
Recording 017
Recording 018
Recording 019
Session 0000
Recording 020
Session 0000
Recording 021
Session 0000
Recording 022
Session 0000
Recording 023
Session 0000
Recording 024
Session 0000
Recording 025
Session 0000
Recording 026
Session 0000
Recording 027
Session 0000
Recording 028
Session 0000
Recording 029
Session 0000
Recording 030
Session 0000
Session

In [13]:
features_mean_video = np.array(list_mean_features_video)
features_var_video = np.array(list_var_features_video)
features_std_video = np.array(list_std_features_video)
print(valid_index)

[0, 1, 2, 3, 4, 5, 6, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 151, 52, 53, 54, 55, 57, 58, 59, 61, 63, 66, 67, 68, 69, 75, 79, 80, 81, 82, 84, 87, 88, 159, 91, 92, 93, 94, 96, 97, 98, 103, 105, 106, 105, 106, 130, 131, 132, 133, 134, 142, 149, 50, 151, 152, 153, 154, 155, 156, 157, 84, 158, 159, 160, 161]


In [14]:
# There's a shrinkage of the dataset between feature_video(110) and feature_app(128)
# The problems can be:
# 1. There are no Session document in Recording but video_data and application_data show the application switching
# 2. Video_data shows there are n totally sessions but only m(m<n) sessions instead

In [15]:
feature_valid_app=[]
feature_valid_switchingTime=[]
valid_label=[]
for index in valid_index:
    feature_valid_app.append(feature_app[index][0])
    feature_valid_switchingTime.append(feature_switchingTime[index])
    valid_label.append(np_label[index])
feature_valid_app = np.array(feature_valid_app).reshape(len(feature_valid_app),1)
valid_label = np.array(valid_label)
print(feature_valid_app.shape)

(101, 1)


In [16]:
valid_label.shape

(101,)

# Feature of app category:

In [17]:
app

['com.android.settings',
 'com.instagram.android',
 'com.miui.securitycenter',
 'com.miui.cleanmaster',
 'com.google.android.googlequicksearchbox',
 'com.whatsapp',
 'com.google.android.youtube',
 'com.spotify.music',
 'com.google.android.gm',
 'com.android.browser',
 'com.Slack',
 'com.facebook.orca',
 'com.android.providers.downloads.ui',
 'ro.bcr.georgego',
 'com.android.camera',
 'com.eTokenBCR',
 'com.facebook.katana',
 'com.android.htmlviewer',
 'com.android.deskclock',
 'com.google.android.apps.maps',
 'com.google.android.gms',
 'com.miui.gallery',
 'com.android.mms',
 'com.android.contacts',
 'com.android.vending',
 'com.revolut.revolut',
 'com.android.chrome']

In [18]:
category = ['financial', 'explorer', 'tools', 'shopping_food_travel',  'sports', 'efficiency', 'entertainment', 'social_media', 'system', 'others']
dict_app = {'financial': ['ro.bcr.georgego','com.eTokenBCR','com.revolut.revolut', 'com.ubs.swidKXJ.android', 'com.ubs.swidK2Y.android',],
                   'explorer': [ 'com.android.chrome','com.android.browser', 'com.android.htmlviewer','com.android.chrome', 'com.UCMobile.intl'],
                   'tools': [ 'com.huawei.search', 'com.android.calendar','com.google.android.apps.maps', 'ch.admin.meteoswiss','com.android.deskclock','com.android.deskclock','com.google.android.apps.maps','com.android.calendar', 'com.weather.Weather'],
                   'shopping_food_travel': [ 'com.wahanda.marketplace', 'com.contextlogic.wish', 'com.fision',  'com.app.tgtg', 'com.airbnb.android'],
                   'sports': [ 'com.espn.score_center', 'de.motain.iliga', 'com.popularapp.thirtydayfitnesschallenge', 'com.bleacherreport.android.teamstream', 'ch.publibike.app', 'homedia.sky.sport'],
                   'efficiency': ['com.google.android.gm', 'com.microsoft.office.word','com.example.android.notepad', 'com.thisclicks.wiw','com.google.android.googlequicksearchbox','com.google.android.gm','com.google.android.gms', 'com.android.email', 'com.google.android.apps.translate', 'cn.wps.moffice_eng'],
                   'entertainment': ['com.apple.android.music','com.google.android.apps.books', 'com.huawei.camera', 'com.google.android.youtube', 'com.adobe.lrmobile','com.huawei.himovie.overseas', 'com.netflix.mediaclient','com.google.android.youtube','com.spotify.music', 'com.android.camera', 'com.miui.player', 'fm.player'],
                   'social_media': ['com.instagram.android', 'com.whatsapp','com.facebook.katana', 'com.android.contacts', 'com.pinterest','com.instagram.android', 'com.whatsapp', 'com.facebook.orca', 'com.facebook.katana', 'com.android.mms','com.android.contacts','jp.naver.line.android',  'com.Slack'],
                   'system': [ 'com.huawei.systemmanager', 'com.ohh2ahh.zuerimensa', 'com.android.incallui','com.android.settings', 'com.huawei.android.internal.app', 'com.android.gallery3d','com.android.vending', 'com.huawei.android.hwouc','com.miui.gallery', 'com.android.settings', 'com.miui.securitycenter', 'com.miui.cleanmaster', 'com.android.providers.downloads.ui', 'com.android.vending',  'com.android.systemui', 'com.android.incallui', 'com.mi.android.globalFileexplorer','android']
                  }

In [19]:
list_feature_app_category = []
for digit in input:
    for key, value in dict_app.items():
        for element in value:
            if element == app[digit]:
                list_feature_app_category.append(category.index(key))
                break
feature_app_category = np.array(list_feature_app_category).reshape(len(list_feature_app_category),1)
feature_app_category.shape

(162, 1)

In [20]:
feature_valid_app_category=[]
for index in valid_index:
    feature_valid_app_category.append(feature_app_category[index][0])
feature_valid_app_category = np.array(feature_valid_app_category).reshape(len(feature_valid_app_category),1)

# Label with categories

In [21]:
list_label_category = []
for digit in label:
    for key, value in dict_app.items():
        for element in value:
            if element== app[digit]:
                list_label_category.append(category.index(key))
                break
label_category = np.array(list_label_category).reshape(len(list_label_category),1)
label_valid_category=[]
for index in valid_index:
    label_valid_category.append(label_category[index][0])
label_valid_category = np.array(label_valid_category).reshape(len(label_valid_category),)
print(label_valid_category)

[7 7 8 8 7 5 8 6 7 6 7 5 1 6 6 8 8 6 6 7 8 8 8 6 5 7 6 8 6 8 8 7 7 0 7 0 6
 7 6 7 8 7 7 8 7 5 1 7 7 7 8 0 7 7 7 1 7 2 5 2 7 8 8 6 6 6 8 8 6 7 6 7 6 7
 7 6 6 6 6 6 6 6 7 7 7 7 6 7 7 1 7 7 6 7 8 7 6 7 6 8 8]


# Feature of app frequency

In [22]:
hash_app_frequency = {}
for digit in input:
    hash_app_frequency[digit] = (hash_app_frequency.get(digit, 0) + 1)
for digit in label:
    hash_app_frequency[digit] = (hash_app_frequency.get(digit, 0) + 1)
    
list_app_frequency = []
for digit in input:
    list_app_frequency.append(hash_app_frequency.get(digit, 0))
feature_app_frequency = np.array(list_app_frequency).reshape(len(list_app_frequency),1)
feature_app_frequency.shape

(162, 1)

In [23]:
feature_valid_app_frequency=[]
for index in valid_index:
    feature_valid_app_frequency.append(feature_app_frequency[index][0])
feature_valid_app_frequency = np.array(feature_valid_app_frequency).reshape(len(feature_valid_app_frequency),1)
print(feature_valid_app_frequency.shape)

(101, 1)


# Feature Duration

In [26]:
feature_duration = np.array(duration).reshape(len(duration),1)
feature_valid_duration = []
for index in valid_index:
    feature_valid_duration.append(feature_duration[index][0])
feature_valid_duration = np.array(feature_valid_duration).reshape(len(feature_valid_duration),1)
print(feature_valid_duration.shape)

(101, 1)


# Feature TimeOfTheDay

In [27]:
# switching time:
feature_switchingTime = []
for element in temp_switchingTime:
    feature_switchingTime.append(int(element))

    
# switching timedate
temp = []
for index in range(len(temp_switchingTimeDate)):
    temp.append(temp_switchingTimeDate[index][11:13]+temp_switchingTimeDate[index][14:16]+temp_switchingTimeDate[index][17:19])
feature_switchingTimeDate = []
for element in temp:
    feature_switchingTimeDate.append(int(element))
tt_feature_switchingTimeDate = np.array(feature_switchingTimeDate).reshape(len(feature_switchingTimeDate),1)
feature_valid_switchingTimeDate = []
for index in valid_index:
    feature_valid_switchingTimeDate.append(tt_feature_switchingTimeDate[index][0])
feature_valid_switchingTimeDate = np.array(feature_valid_switchingTimeDate).reshape(len(feature_valid_switchingTimeDate),1)
print(feature_valid_switchingTimeDate.shape)

(101, 1)


In [49]:
feature_valid_switchingTimeDate

array([[221951],
       [222017],
       [222517],
       [222529],
       [222823],
       [222900],
       [222907],
       [ 81009],
       [ 81015],
       [ 81211],
       [ 81316],
       [ 81529],
       [ 81633],
       [ 81640],
       [ 91800],
       [ 91837],
       [ 91838],
       [ 91852],
       [ 91933],
       [ 92002],
       [183625],
       [183630],
       [183834],
       [183836],
       [162920],
       [163535],
       [163537],
       [163540],
       [163544],
       [163612],
       [163616],
       [163617],
       [184659],
       [184704],
       [201008],
       [200304],
       [200313],
       [200402],
       [200402],
       [201555],
       [ 81814],
       [ 83010],
       [ 83244],
       [ 90603],
       [112658],
       [112740],
       [155205],
       [192505],
       [192631],
       [192636],
       [173051],
       [201400],
       [212930],
       [155549],
       [162803],
       [163110],
       [ 81133],
       [ 82035],
       [ 82235

# Feature DeviceStatus(3/5/10)

In [28]:
deviceStatus3=[0]*len(feature_switchingTime)
deviceStatus5=[0]*len(feature_switchingTime)
deviceStatus10=[0]*len(feature_switchingTime)
for recording in os.listdir(account_path):
    recording_path = os.path.join (account_path, str(recording))
    
    assert(os.path.exists(recording_path))


    # open each of the recording file
    
    with open (os.path.join(recording_path, "video_data.txt")) as video_file:
        video_data = json.load(video_file)["video_data"]
        #num_session = len(video_data)
        #????????????????????????????????????????????? something wrong with the video_data for recording 001&002
        
            
        with open(os.path.join(recording_path, "device_data.txt")) as device_file:
            device_data = json.load(device_file)["device_data"]

            for d_index in range(len(device_data)):
                for s_index in range(len(feature_switchingTime)):
                    if device_data[d_index]["time"] >= feature_switchingTime[s_index] - 3*1_000_000_000 and device_data[d_index]["time"] <= feature_switchingTime[s_index]:
                        deviceStatus3[s_index]+=1
                    if device_data[d_index]["time"] >= feature_switchingTime[s_index] - 5*1_000_000_000 and device_data[d_index]["time"] <= feature_switchingTime[s_index]:
                        deviceStatus5[s_index]+=1                    
                    if device_data[d_index]["time"] >= feature_switchingTime[s_index] - 10*1_000_000_000 and device_data[d_index]["time"] <= feature_switchingTime[s_index]:
                        deviceStatus10[s_index]+=1            
print(len(deviceStatus3))

162


In [29]:
feature_deviceStatus3 = np.array(deviceStatus3).reshape(len(deviceStatus3),1)
feature_valid_deviceStatus3 = []
for index in valid_index:
    feature_valid_deviceStatus3.append(feature_deviceStatus3[index][0])
feature_valid_deviceStatus3 = np.array(feature_valid_deviceStatus3).reshape(len(feature_valid_deviceStatus3),1)
print(feature_valid_deviceStatus3.shape)

(101, 1)


# Feature TouchEvent(3/5/10)

In [30]:
# touch data: happened touch events in the past 3/5/10 seconds
touchTimes_data3=[0]*len(feature_switchingTime)
touchTimes_data5=[0]*len(feature_switchingTime)
touchTimes_data10=[0]*len(feature_switchingTime)
for recording in os.listdir(account_path):
    recording_path = os.path.join (account_path, str(recording))
    
    assert(os.path.exists(recording_path))


    # open each of the recording file
    
    with open (os.path.join(recording_path, "video_data.txt")) as video_file:
        video_data = json.load(video_file)["video_data"]
        #num_session = len(video_data)
        #????????????????????????????????????????????? something wrong with the video_data for recording 001&002
        

        for file in os.listdir(recording_path):
            if str(file)[:7] == "Session":
                # open each of the session file
                session_path = os.path.join(recording_path, str(file))

                assert(os.path.exists(session_path))

                with open(os.path.join(session_path, "touch_data.txt")) as touch_file:
                    touch_data = json.load(touch_file)["touch_data"]

                    for t_index in range(len(touch_data)):
                        for s_index in range(len(feature_switchingTime)):
                            if touch_data[t_index]["time"] >= feature_switchingTime[s_index] - 3*1_000_000_000 and touch_data[t_index]["time"] <= feature_switchingTime[s_index]:
                                touchTimes_data3[s_index]+=1
                            if touch_data[t_index]["time"] >= feature_switchingTime[s_index] - 5*1_000_000_000 and touch_data[t_index]["time"] <= feature_switchingTime[s_index]:
                                touchTimes_data5[s_index]+=1                    
                            if touch_data[t_index]["time"] >= feature_switchingTime[s_index] - 10*1_000_000_000 and touch_data[t_index]["time"] <= feature_switchingTime[s_index]:
                                touchTimes_data10[s_index]+=1            
print(len(touchTimes_data3))

162


In [31]:
feature_touchTimes3 = np.array(touchTimes_data3).reshape(len(touchTimes_data3),1)
feature_valid_touchTimes3 = []
for index in valid_index:
    feature_valid_touchTimes3.append(feature_touchTimes3[index][0])
feature_valid_touchTimes3 = np.array(feature_valid_touchTimes3).reshape(len(feature_valid_touchTimes3),1)
print(feature_valid_touchTimes3.shape)

(101, 1)


# valid Label

In [32]:
label_app = np.array(label).reshape(len(label),1)
label_valid_app=[]
for index in valid_index:
    label_valid_app.append(label_app[index][0])
label_valid_app = np.array(label_valid_app).reshape(len(label_valid_category),)
print(label_valid_app.shape)

(101,)


# Function

In [33]:
def predict_top_k(X_test, y_test, model, k=3):

    probs = model.predict_proba(X_test)
    labels = model.classes_
    result = 0

    for i in range(probs.shape[0]):
        a = probs[i]
        index = heapq.nlargest(k, range(probs.shape[1]), a.take)
        for element in index:
            if labels[element] == y_test[i]:
                result+=1
    
    return result/y_test.shape[0]

def kNN_training (feature, label):
    valid_X = scaler.fit_transform(feature)
    valid_y = label
    X_train, X_test, y_train, y_test = train_test_split(valid_X, valid_y, random_state=0)
    model = KNeighborsClassifier(n_neighbors=7, weights='uniform').fit(X_train,y_train)
    accuracy = model.score(X_test, y_test)
    top_accuracy4 = predict_top_k(X_test, y_test, model, k=4)
    top_accuracy8 = predict_top_k(X_test, y_test, model, k=8)
    return accuracy, top_accuracy4, top_accuracy8

def rf_training (feature, label):
    valid_X = scaler.fit_transform(feature)
    valid_y = label
    X_train, X_test, y_train, y_test = train_test_split(valid_X, valid_y, random_state=0)
    model = RandomForestClassifier(n_estimators=50).fit(X_train,y_train)
    accuracy = model.score(X_test, y_test)
    top_accuracy4 = predict_top_k(X_test, y_test, model, k=4)
    top_accuracy8 = predict_top_k(X_test, y_test, model, k=8)
    return accuracy, top_accuracy4, top_accuracy8

def Ada_training (feature, label):
    valid_X = scaler.fit_transform(feature)
    valid_y = label
    X_train, X_test, y_train, y_test = train_test_split(valid_X, valid_y, random_state=0)
    model = AdaBoostClassifier(n_estimators=100, random_state=0).fit(X_train,y_train)
    accuracy = model.score(X_test, y_test)
    top_accuracy4 = predict_top_k(X_test, y_test, model, k=4)
    top_accuracy8 = predict_top_k(X_test, y_test, model, k=8)
    return accuracy, top_accuracy4, top_accuracy8

def dummy_training(feature, label):
    dummy = DummyClassifier(strategy="most_frequent")
    dummy = dummy.fit(feature, label)
    accuracy = dummy.score(feature, label)
    return accuracy

def calculateMeanRF(feature, label):
    accuracy=[]
    top4=[]
    top8=[]
    for i in range(5):
        accuracy_rf, top_accuracy_rf4, top_accuracy_rf8= rf_training (feature, label)
        accuracy.append(accuracy_rf)
        top4.append(top_accuracy_rf4)
        top8.append(top_accuracy_rf8)
    return sum(accuracy)/len(accuracy), sum(top4)/len(top4), sum(top8)/len(top8)

def calculateMeanKNN(feature, label):
    accuracy=[]
    top4=[]
    top8=[]
    for i in range(5):
        accuracy_kNN, top_accuracy_kNN4, top_accuracy_kNN8= kNN_training (feature, label)
        accuracy.append(accuracy_kNN)
        top4.append(top_accuracy_kNN4)
        top8.append(top_accuracy_kNN8)
    return sum(accuracy)/len(accuracy), sum(top4)/len(top4), sum(top8)/len(top8)

def calculateMeanAda(feature, label):
    accuracy=[]
    top4=[]
    top8=[]
    for i in range(5):
        accuracy_kNN, top_accuracy_kNN4, top_accuracy_kNN8= Ada_training (feature, label)
        accuracy.append(accuracy_kNN)
        top4.append(top_accuracy_kNN4)
        top8.append(top_accuracy_kNN8)
    return sum(accuracy)/len(accuracy), sum(top4)/len(top4), sum(top8)/len(top8)

# Features Summary

In [34]:
print(feature_valid_app.shape)
print(feature_valid_duration.shape)
print(feature_valid_switchingTimeDate.shape)
print(feature_valid_deviceStatus3.shape)
print(feature_valid_touchTimes3.shape)
print(feature_valid_app_category.shape)
print(feature_valid_app_frequency.shape)
print(features_mean_video.shape)
print(features_var_video.shape)
print(features_std_video.shape)
print(label_valid_category.shape)
print(label_valid_app.shape)

(101, 1)
(101, 1)
(101, 1)
(101, 1)
(101, 1)
(101, 1)
(101, 1)
(101, 8)
(101, 8)
(101, 8)
(101,)
(101,)


In [43]:
allFeatures = np.concatenate([feature_valid_app,
                              feature_valid_duration,
                              feature_valid_switchingTimeDate,
                              feature_valid_deviceStatus3,
                              feature_valid_touchTimes3,
                              feature_valid_app_category, 
                              feature_valid_app_frequency, 
                              features_mean_video, 
                              features_var_video,
                              features_std_video],
                              axis=1)
print(allFeatures.shape)

(101, 31)


# Feature Selection

In [44]:
#L1-based feature selection for all 288+1 features
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC

linearsvc = LinearSVC(C=0.01, penalty='l1', dual=False).fit(allFeatures, label_valid_app)
model = SelectFromModel(linearsvc, prefit=True)
X_new_l1 = model.transform(allFeatures)
X_new_l1.shape

D:\Softwares\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


(101, 3)

In [45]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
clf = ExtraTreesClassifier(n_estimators=100).fit(allFeatures, label_valid_app)
model = SelectFromModel(clf, prefit=True)
X_new_tree = model.transform(allFeatures)
X_new_tree.shape

(101, 9)

# Model Performance

In [46]:
accuracy_rf, top_accuracy_rf4, top_accuracy_rf8 = calculateMeanRF(X_new_tree, label_valid_app)
print(accuracy_rf, top_accuracy_rf4, top_accuracy_rf8)

0.3 0.6230769230769232 0.7846153846153845


In [47]:
accuracy_kNN, top_accuracy_kNN4, top_accuracy_kNN8 = calculateMeanKNN(X_new_tree, label_valid_app)
print(accuracy_kNN, top_accuracy_kNN4, top_accuracy_kNN8)

0.15384615384615385 0.5 0.6538461538461539


In [48]:
accuracy_ada, top_accuracy_ada4, top_accuracy_ada8 = calculateMeanAda(X_new_tree, label_valid_app)
print(accuracy_ada, top_accuracy_ada4, top_accuracy_ada8)

0.19230769230769232 0.4230769230769231 0.7692307692307693


# ========================================================

In [ ]:
# 1. probabilistic model                            check
# 2. columns manuelly select                        check
# 3. mean, variance, standard deviation             check
# 4. weights of the frequency                       check
# 5. categorical app                                check

In [55]:
gaze = pd.read_csv(r"E:\37be2146-0a9a-4876-aef9-89a771363418\37be2146-0a9a-4876-aef9-89a771363418\Recording 001\Session 0002\r_video.csv")
gaze.iloc[:,3:]

,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,gaze_1_z,gaze_angle_x,gaze_angle_y,...,eye_lmk_Z_46,eye_lmk_Z_47,eye_lmk_Z_48,eye_lmk_Z_49,eye_lmk_Z_50,eye_lmk_Z_51,eye_lmk_Z_52,eye_lmk_Z_53,eye_lmk_Z_54,eye_lmk_Z_55
0,0.98,1,-0.058954,-0.457938,-0.887027,-0.135234,-0.456709,-0.879277,-0.110,-0.478,...,416.0,416.9,419.5,419.7,420.5,421.4,422.0,421.8,421.0,420.1
1,0.98,1,-0.053291,-0.443624,-0.894627,-0.143240,-0.442300,-0.885355,-0.110,-0.462,...,426.4,427.4,429.6,429.7,430.4,431.3,431.8,431.6,430.9,430.1
2,0.98,1,-0.074696,-0.357094,-0.931077,-0.129029,-0.425962,-0.895493,-0.111,-0.405,...,423.7,424.7,426.9,427.0,427.6,428.5,429.0,429.0,428.3,427.4
3,0.93,1,-0.042742,-0.400226,-0.915419,-0.191986,-0.352971,-0.915725,-0.127,-0.390,...,435.4,436.6,438.9,439.1,439.7,440.4,440.9,440.8,440.1,439.4
4,0.03,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,...,369.3,370.0,372.3,372.9,373.7,374.4,374.5,373.9,373.1,372.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,0.03,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,...,256.7,255.0,259.8,260.3,261.9,263.9,264.9,264.5,262.8,260.9
370,0.03,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,...,258.0,256.2,261.0,261.5,263.1,265.0,266.1,265.6,263.9,262.0
371,0.03,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,...,264.3,262.6,267.3,267.7,269.4,271.3,272.4,271.9,270.3,268.3
372,0.03,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,...,259.0,257.3,261.9,262.4,264.1,266.0,267.0,266.6,264.9,263.0
